## IMPORT ALL THE NECESSARY LIBRARIES FOR THE PROJECT

In [6]:
!pip install opencv-python
!pip install torch
!pip install torchvision

   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   -------------------------------- ------- 1.0/1.3 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 8.2 MB/s eta 0:00:00


In [44]:
import os
import csv
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, utils, io
from torchvision.utils import make_grid
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

from string import ascii_lowercase
from tqdm import tqdm # For progress bar

In [45]:
# Setting the path of the training dataset (that was already provided to you)

running_local = True if os.getenv('JUPYTERHUB_USER') is None else False
DATASET_PATH = "."

# Set the location of the dataset
if running_local:
    # If running on your local machine, the sign_lang_train folder's path should be specified here
    local_path = os.path.join('..', 'sign_lang_train')
    if os.path.exists(local_path):
        DATASET_PATH = local_path
else:
    # If running on the Jupyter hub, this data folder is already available
    # You DO NOT need to upload the data!
    DATASET_PATH = "/data/mlproject22/sign_lang_train"

### 1.2 Data Loading using PyTorch

In [ ]:
### LETS READ THE CSV FILE AND THE .JPG FILES WITH THE CLASS BUILT IN SRC ###
from datasets import SignLangDataset
csv_filename = "labels.csv"  # This is your file inside sign_lang_train

# Create dataset
dataset = SignLangDataset(csv_file=csv_filename, root_dir=DATASET_PATH)

In [31]:
### DID WE READ IT CORRECTLY??? ###
dataset.__len__()

9680

In [11]:
### AFTER MAKING SURE THAT THE DATASET IS CORRECTLY UPLOADED ###
### SPLIT THE DATASET INTO TRAINING AND VALIDATION SETS ###
# 80/20 split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [12]:
### CREATE THE DATALOADERS TO LOAD THE DATA IN BATCHES OR TO SHUFFLE IT IF NEEDED ###
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [ ]:
### RANDOM FOREST MODEL IS OUR SECOND OPTION ###

# 1. Feature extraction transform
class FlattenImageTransform:
    def __init__(self, target_size=(64, 64)):
        self.target_size = target_size

    def __call__(self, sample):
        image = sample['image'] # Shape (1, H, W)
        label = sample['label']

        # Convert torch tensor to numpy array if necessary for cv2
        # Your __getitem__ already returns numpy, so this might not be strictly needed,
        # but it's good practice if you introduce other torch transforms later.
        if isinstance(image, torch.Tensor):
            image = image.squeeze(0).numpy() # Remove channel dim, convert to numpy (H, W)

        # Resize the image (important for consistent feature vector length)
        # Interpolation method for resizing grayscale images
        image_resized = cv2.resize(image, self.target_size, interpolation=cv2.INTER_AREA)

        # Flatten the image into a 1D feature vector
        features = image_resized.flatten() # Shape (target_size[0] * target_size[1],)

        return {'features': features, 'label': label}
    
# For Flattened Pixels:
transform_for_rf = FlattenImageTransform(target_size=(64, 64))

# Or for HOG:
# transform_for_rf = HOGFeatureTransform(target_size=(64, 64))


# Re-create the dataset instances with the new transform
# NOTE: Your current SignLangDataset doesn't directly take `transform` in __getitem__
# You commented out `if self.transform: sample = self.transform(sample)`
# You need to uncomment/implement that or apply the transform manually as shown below.

# Let's assume you'll apply it manually by processing the data loaders:

# --- Manually apply transform and collect features/labels ---
X_train_features = []
y_train_labels = []
X_val_features = []
y_val_labels = []

print("Extracting features from training data...")
for i, sample in enumerate(tqdm(train_dataset)):
    processed_sample = transform_for_rf(sample)
    X_train_features.append(processed_sample['features'])
    y_train_labels.append(processed_sample['label'])

print("Extracting features from validation data...")
for i, sample in enumerate(tqdm(val_dataset)):
    processed_sample = transform_for_rf(sample)
    X_val_features.append(processed_sample['features'])
    y_val_labels.append(processed_sample['label'])

# Convert lists to numpy arrays
X_train = np.array(X_train_features)
y_train = np.array(y_train_labels)
X_val = np.array(X_val_features)
y_val = np.array(y_val_labels)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_val: {X_val.shape}")
print(f"Shape of y_val: {y_val.shape}")


Extracting features from training data...


100%|██████████| 7744/7744 [00:09<00:00, 853.77it/s] 


Extracting features from validation data...


100%|██████████| 1936/1936 [00:02<00:00, 849.09it/s]


Shape of X_train: (7744, 524288)
Shape of y_train: (7744,)
Shape of X_val: (1936, 524288)
Shape of y_val: (1936,)


In [17]:
# 2. Train the Random Forest Classifier
print("\nTraining Random Forest Classifier...")
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1) # n_jobs=-1 uses all available cores
rf_classifier.fit(X_train, y_train)
print("Training complete.")

# 3. Evaluate the model
print("\nEvaluating the model on the validation set...")
y_pred = rf_classifier.predict(X_val)

accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

# Optional: Print a more detailed classification report
print("\nClassification Report:")
# You'll need the original class names for a clearer report
# Assuming your dataset.class_names is available
# For example: class_names = list(map(str, list(range(10)))) + list(ascii_lowercase)
# from string import ascii_lowercase
# full_class_names = list(map(str, list(range(10)))) + list(ascii_lowercase)
print(classification_report(y_val, y_pred, target_names=dataset.class_names))

# You can also save your model if you wish
# import joblib
# joblib.dump(rf_classifier, 'random_forest_model.joblib')
# print("Random Forest model saved.")


Training Random Forest Classifier...
Training complete.

Evaluating the model on the validation set...
Validation Accuracy: 0.7237

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.91      0.82       128
           1       1.00      0.21      0.35        28
           2       0.50      0.04      0.07        25
           3       0.80      0.21      0.33        19
           4       0.63      0.89      0.74       117
           5       0.62      0.50      0.55        16
           6       0.59      0.79      0.67       112
           7       0.00      0.00      0.00        20
           8       1.00      0.14      0.24        37
           9       0.72      0.94      0.81       114
           a       0.73      0.38      0.50        21
           b       0.90      0.87      0.89        54
           c       0.85      0.93      0.89       100
           d       0.80      0.36      0.50        33
           e       1.00      0.35

c:\Users\frala\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\frala\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\frala\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [18]:
import joblib

# --- Saving the model ---
model_filename = 'random_forest_sign_lang_model.joblib'
joblib.dump(rf_classifier, model_filename) # Replace rf_model with rf_model_hog or rf_model_deep if you used those
print(f"Model saved to {model_filename}")

# --- Loading the model later (in a new script or session) ---
# loaded_rf_model = joblib.load(model_filename)
# print("Model loaded successfully!")

# --- Example of using the loaded model for prediction (if you load it) ---
# new_image_features = ... # Your new image, preprocessed using the SAME feature extraction method
# prediction = loaded_rf_model.predict(new_image_features)
# predicted_class_name = dataset.class_names[prediction[0]] # Assuming prediction is an array
# print(f"Predicted class for new image: {predicted_class_name}")

Model saved to random_forest_sign_lang_model.joblib
